In [1]:
import seaborn as sns
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
%pylab inline
import os
from __future__ import print_function
from __future__ import division
import os, sys
# hook up all scripts
sys.path.append(os.path.abspath('../scripts')) # if notebook in folder
sys.path.append(os.path.abspath('./scripts')) #if in root
import preprocessing_time_series as pts


os.chdir(pts.working_dir())

Populating the interactive namespace from numpy and matplotlib


In [2]:
data_train = pd.read_csv('Feature_releases/release_v01/train_QUERETARO.csv')
print(data_train.shape)
data_train.head()

(319693, 24)


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,No_remains,Venta_hoy_by_uni,Ordered,Log_Demanda,Median_Producto_ID,LogMean_Producto_ID,Median_Producto_ID_Ruta_SAK,LogMean_Producto_ID_Ruta_SAK,Median_Producto_ID_Cliente_ID_Agencia_ID,LogMean_Producto_ID_Cliente_ID_Agencia_ID
0,3,2059,7,3301,237195,202,8,327.60,0,0.0,...,0,40.95,1,2.197225,7.0,2.217120,12.0,2.436608,8.0,2.246207
1,3,2059,7,3301,237195,1150,2,27.92,0,0.0,...,0,13.96,1,1.098612,2.0,1.471009,4.0,1.601341,2.0,0.993963
2,3,2059,7,3301,237195,1240,4,33.52,0,0.0,...,0,8.38,1,1.609438,3.0,1.488157,5.0,2.428383,5.0,2.206691
3,3,2059,7,3301,237195,1250,5,38.20,0,0.0,...,0,7.64,1,1.791759,4.0,1.766284,6.0,2.531459,5.0,1.791759
4,3,2059,7,3301,237195,2233,2,39.88,0,0.0,...,0,19.94,1,1.098612,2.0,1.432443,8.0,2.214223,2.0,1.418451


In [3]:
town = pts.text_encoding(pts.town_preproc())
agencies = set(town.loc[town.State == u'NAYARIT'].index)
len(agencies)

Data endoded


10

In [4]:
data_test = pd.read_csv('test.csv', index_col=0)
data_test = data_test.loc[data_test.Agencia_ID.isin(agencies), :]
print(data_test.shape)
data_test.head()

(78819, 6)


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
id,,,,,,
42,11,1378,1,2005,594640,43285
59,10,1359,11,3957,2068213,6469
129,10,1378,1,2101,905734,37577
183,11,1340,1,2065,325592,43285
203,11,1340,1,2052,7729188,43274


In [5]:
data = pd.concat([data_train, data_test], axis=0)
del data_train
del data_test

In [6]:
data = pts.lag_generation(data, n_lags=5)
data = data.drop([u'Town', u'State', u'Venta_uni_hoy', u'Venta_hoy', u'Dev_uni_proxima',
       u'Dev_proxima', u'Demanda_uni_equil', u'Dev_proxima_by_uni', u'No_remains', 
        u'Venta_hoy_by_uni', u'Ordered',u'Median_Producto_ID',
       u'LogMean_Producto_ID', u'Median_Producto_ID_Ruta_SAK',
       u'LogMean_Producto_ID_Ruta_SAK',
       u'Median_Producto_ID_Cliente_ID_Agencia_ID',
       u'LogMean_Producto_ID_Cliente_ID_Agencia_ID'], axis=1).set_index(u'Semana')
# data.describe().transpose()

1 lag done
2 lag done
3 lag done
4 lag done
5 lag done


In [7]:
X_train = data.loc[~data.Log_Demanda.isnull(),:].drop('Log_Demanda', axis=1)
y_train = data.loc[~data.Log_Demanda.isnull(),'Log_Demanda']
X_eval = data.loc[data.Log_Demanda.isnull(),:]

In [8]:
import crossval

cv5fold, strats = crossval.crossvalidation(X_train[['Canal_ID']])
strats.value_counts()

1     290404
11     21707
7       4422
2       3027
8        133
dtype: int64

# xgboost 

Обнаружил, что для learning_rate=0.3 оптимальное количество деревьев 78.

Сейчас подбираю параметры max_depth и min_child_weight.

Потом надо подобрать [gamma], [subsample, colsample_bytree], [lambda(L2), alpha(L1)]. 

In [9]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score

def plot_feature_importances(xgb_model):
    plt.figure(figsize=(20,10))
    feat_imp = pd.Series(xgb_model.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
grid_param = {
 'max_depth': [15, 17, 20, 23, 25],
 'min_child_weight': [5, 8, 10, 12],
}

xgb_model = xgb.XGBRegressor(n_estimators=78,
                             learning_rate = 0.3,
                             gamma=0,
                             subsample=0.8,
                             colsample_bytree=0.8)

clf = GridSearchCV(xgb_model, grid_param, cv=cv5fold,
                   scoring='mean_squared_error',verbose=1, n_jobs=-1)
clf.fit(X_train, y_train)

print(clf.grid_scores_)
print(clf.best_score_)
print(clf.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
plot_feature_importances(clf.best_estimator_)

In [ ]:
# param = clf.best_params_
param = {
 'gamma': 0,
 'max_depth': 15,
 'min_child_weight': 6,
 'n_estimators': 100,
 'subsample': 0.8
}

xgb_model = xgb.XGBRegressor()
xgb_model.set_params(**param)
scores = cross_val_score(xgb_model, X_train, y_train, scoring = 'mean_squared_error',
                             cv = cv5fold, n_jobs=-1)
xgb_model.fit(X_train, y_train)

In [ ]:
plot_feature_importances(xgb_model)
r2 = xgb_model.score(X_train, y_train)

rmse = np.sqrt((1-r2)*y_train.var())
print (scores, rmse)

In [ ]:
param = {
 'eta': 0.3
 'gamma': 0,
 'max_depth': 15,
 'min_child_weight': 6,
 'subsample': 0.8
}

xgtrain = xgb.DMatrix(X_train.values, label=y_train.values)
cvresult = xgb.cv(param, xgtrain, num_boost_round=200, nfold=5,
                    metrics='rmse', early_stopping_rounds=10)
cvresult